In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 6.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import re
import nltk
import emoji
import string
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import TweetTokenizer

In [ ]:
data = pd.read_csv('/content/gdrive/MyDrive/tweet_data.csv')

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
lemmatizer = WordNetLemmatizer()
tk = TweetTokenizer()

In [ ]:
stopwords = ["i", "am", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself",
        "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself",
        "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that",
        "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had",
        "having", "do", "does", "did", "doing", "a", "an", "the", "and", "if", "or", "because", "as",
        "until", "while", "of", "at", "by", "for", "with", "about", "between", "into", "through",
        "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off",
        "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all",
        "any", "both", "each", "other", "such", "own", "same", "so", "than", "s", "t", "can", "will", "just", "now"]

punctuations = ["+", ",", ".", "-", "\\", "&", "!", "?", ":", ";", "#", "~", "=", "/", "$", "£", "^", "(", ")", "_", "<", ">", "%"]

In [ ]:
dict_smileys = {
        ":-)":"smiley", ":-]":"smiley", ":-3":"smiley", ":->":"smiley", "8-)":"smiley", ":-}":"smiley", ":)":"smiley", ":]":"smiley",
        ":3":"smiley", ":>":"smiley", "8)":"smiley", ":}":"smiley", ":o)":"smiley", ":c)":"smiley", ":^)":"smiley", "=]":"smiley",
        "=)":"smiley", ":-))":"smiley", ":‑D":"smiley", "8‑D":"smiley","x‑D":"smiley","X‑D":"smiley",":D":"smiley", "8D":"smiley","xD":"smiley",
        "XD":"smiley", ":‑(":"sad", ":‑c":"sad",":‑<":"sad",":‑[":"sad",":(":"sad",":c":"sad",":<":"sad",":[":"sad", ":-||":"sad", ">:[":"sad",
        ":{":"sad", ":@":"sad", ">:(":"sad", ":'‑(":"sad", ":'(":"sad", ":‑P":"playful", "X‑P":"playful", "x‑p":"playful", ":‑p":"playful",
        ":‑Þ":"playful", ":‑þ":"playful", ":‑b":"playful", ":P":"playful", "XP":"playful", "xp":"playful",":p":"playful",":Þ":"playful",
        ":þ":"playful", ":b":"playful", "<3":"love"
        }


dict_contractions = {
        "ain't":"is not", "amn't":"am not", "aren't":"are not", "can't":"cannot", "'cause":"because", "couldn't":"could not","couldn't've":"could not have",
        "could've":"could have","daren't":"dare not","daresn't":"dare not","dasn't":"dare not","didn't":"did not","doesn't":"does not","don't":"do not",
        "e'er":"ever","em":"them","everyone's":"everyone is","finna":"fixing to","gimme":"give me","gonna":"going to","gon't":"go not","gotta":"got to",
        "hadn't":"had not","hasn't":"has not","haven't":"have not","he'd":"he would","he'll":"he will","he's":"he is","he've":"he have",
        "how'd":"how would","how'll":"how will", "how're":"how are", "how's":"how is", "I'd":"I would", "I'll":"I will",
        "I'm":"I am","I'm'a":"I am about to","I'm'o":"I am going to","isn't":"is not",
        "it'd":"it would","it'll":"it will","it's":"it is","I've":"I have","kinda":"kind of","let's":"let us","mayn't":"may not",
        "may've":"may have","mightn't":"might not","might've":"might have","mustn't":"must not", "mustn't've":"must not have", "must've":"must have",
        "needn't":"need not","ne'er":"never", "o'":"of", "o'er":"over", "ol'":"old", "oughtn't":"ought not", "shalln't":"shall not", "shan't":"shall not",
        "she'd":"she would","she'll":"she will","she's":"she is","shouldn't":"should not","shouldn't've":"should not have","should've":"should have",
        "somebody's":"somebody is", "someone's":"someone is", "something's":"something is", "that'd":"that would", "that'll":"that will", "that're":"that are",
        "that's":"that is","there'd":"there would","there'll":"there will","there're":"there are","there's":"there is","these're":"these are",
        "they'd":"they would","they'll":"they will","they're":"they are","they've":"they have","this's":"this is","those're":"those are", "'tis":"it is",
        "'twas":"it was", "wanna":"want to", "wasn't":"was not", "we'd":"we would", "we'd've":"we would have", "we'll":"we will", "we're":"we are",
        "weren't":"were not",
        "we've":"we have","what'd":"what did","what'll":"what will","what're":"what are","what's":"what is","what've":"what have","when's":"when is",
        "where'd":"where did","where're":"where are","where's":"where is","where've":"where have","which's":"which is","who'd":"who would",
        "who'd've":"who would have","who'll":"who will","who're":"who are","who's":"who is","who've":"who have","why'd":"why did","why're":"why are",
        "why's":"why is","won't":"will not","wouldn't":"would not","would've":"would have","y'all":"you all","you'd":"you would",
        "you'll":"you will", "you're":"you are","you've":"you have","Whatcha":"What are you","luv":"love","sux":"sucks"
        }

In [ ]:
def tweet_preprocess(tweet):
    tweet = emoji.demojize(tweet)
    tweet = tweet.lower()   #Converting to Lowercase
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet)  #Removing URLs
    tweet = re.sub(r'^RT[\s]+', '', tweet)   #Removing re-tweet text
    tweet = re.sub(r'\b\w\b', '', tweet) #Removing Single Letters
    tweet = re.sub(r'[\@\#]', '', tweet)  #Removing Hastags and targets (@)
    tweet = re.sub(r'\d+', '', tweet)  #Removing Numbers
    tweet = re.sub(r'\s+', ' ', tweet).strip()  # Removing extra spaces
    for smiley, meaning in dict_smileys.items(): #Emoticons
        tweet = tweet.replace(smiley, meaning)
    for contraction, expanded in dict_contractions.items():  #Contractions
        tweet = tweet.replace(contraction, expanded)
    tweet = tk.tokenize(tweet)  #Tokenizing the tweet
    pos_tags = pos_tag(tweet)
    tweet = [word for word in tweet if word not in stopwords and word not in punctuations]  #Removing Stop words and Punctuations
    final_tweet=[]
    for token,tag in pos_tag(tweet):   #Lemmatizing the tweet
        if token.isdigit():
            continue

        pos=tag[0].lower()
        if pos not in ['a', 'r', 'n', 'v']:
            pos='n'
        final_tweet.append(lemmatizer.lemmatize(token,pos))

    return ' '.join(final_tweet)

In [ ]:
string = '#INDEXATION #Budget2024 The removal of indexation and reduction of LTCG to 12.50% is indeed beneficial to everyone 🙂 .. or say 90% of the people. Below is the example https://t.co/VRqKNvbc2w'
string = tweet_preprocess(string)
print(string)

indexation budget rthemoval indexation reduction ltcg % indeed beneficial everyone slightly_smiling_face .. say % people example


In [ ]:
data['processed_text'] = data['Text'].apply(tweet_preprocess)

In [ ]:
vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(data['processed_text']).toarray()
terms = vectorizer.get_feature_names_out()

In [ ]:
class NMF:
    def __init__(self, n_components, max_iter=200, tol=1e-4):
        self.n_components = n_components
        self.max_iter = max_iter
        self.tol = tol

    def fit_transform(self, X):
        n_samples, n_features = X.shape

        # Initialize W and H matrices with random non-negative values
        np.random.seed(42)
        self.W = np.random.rand(n_samples, self.n_components)
        self.H = np.random.rand(self.n_components, n_features)

        for _ in range(self.max_iter):
            # Update H
            numerator = np.dot(self.W.T, X)
            denominator = np.dot(np.dot(self.W.T, self.W), self.H)
            self.H *= numerator / (denominator + 1e-10)

            # Update W
            numerator = np.dot(X, self.H.T)
            denominator = np.dot(np.dot(self.W, self.H), self.H.T)
            self.W *= numerator / (denominator + 1e-10)

            # Check for convergence
            error = np.linalg.norm(X - np.dot(self.W, self.H))
            if error < self.tol:
                break

        return self.W, self.H

In [ ]:
n_topics = 6
nmf = NMF(n_components=n_topics)
W, H = nmf.fit_transform(X)

In [ ]:
def print_top_terms(H, terms, n_top_terms=6):
    for topic_idx, topic in enumerate(H):
        top_term_indices = topic.argsort()[-n_top_terms:][::-1]
        top_terms = [terms[i] for i in top_term_indices]
        print(f"Topic {topic_idx + 1}: {', '.join(top_terms)}")

In [ ]:
print_top_terms(H, terms)

Topic 1: class, middle, budget, nsitharaman, narendramodi, not
Topic 2: tax, pay, income, budget, like, more
Topic 3: union, budgetforviksitbharat, via, namo, app, budget
Topic 4: youth, union, themployment, unionbudget, budget, job
Topic 5: india, unionbudget, crore, growth, post, lakh
Topic 6: budget, union, finance, not, say, rahulgandhi


In [ ]:
tweet_topics = np.argmax(W, axis=1)
data['topic'] = tweet_topics

In [ ]:
for i in range(5):
    print(f"Document: {data['Text'].iloc[i][:100]}...")
    print(f"Topic: {data['topic'].iloc[i] + 1}\n")

Document: Working overtime so I can pay more taxes. Living the dream, one paycheck at a time.
#Budget2024...
Topic: 2

Document: To show fake numbers of jobs having been created, have companies, MNCs been told to float fake vacan...
Topic: 5

Document: Union Budget 2024: A blueprint for sustainable growth and development
https://t.co/QkM6i1qhID via Na...
Topic: 3

Document: Today he has issues with big businesses!

Truth is it's only PM @narendramodi who has taken the Indi...
Topic: 6

Document: People's leader LOP Shri @RahulGandhi Ji exposed Defence Minister Rajnath Singh’s lies about compens...
Topic: 6

